Mount Google Drive.

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set the path for the output files and data.

In [93]:
my_path = "/content/drive/MyDrive/StudentsResearch/SamuelPaulGyamfi/conjecturing/real_estate/"

Copy the expressions and conjecturing.py file over.

In [94]:
!cp /content/drive/MyDrive/conjecturing/c/build/expressions {my_path}
#!cp /content/drive/MyDrive/conjecturing/python/conjecturing.py {my_path}
!chmod +x {my_path}/expressions

Add the path to the system path so that Python can find the conjecturing executable (expressions)

In [95]:
import sys
sys.path.insert(0, my_path)

Import libraries, specify settings, and prepare data.

In [97]:
import pandas as pd
import numpy as np
import sys
from conjecturing import *
from sklearn.model_selection import train_test_split

# Preprocessing:
# 1. Ensure that column names have no special characters other than "_".
# 2. No column name (other than the target) should have "TARGET" in
#    the name.
# 3. Define inv_file, prop_file, my_data, invariant_names,
#    categorical_names, target, num_train, my_skips, my_time,
#    complexity, use_operators.  Optionally, define debug and verbose.
# 4. Rename the target column TARGET in my_data.

inv_fname = "2025_07_10_inv.txt"
prop_fname = "2025_07_10_prop.txt"

my_time = 5
complexity = 10
num_train = 10
my_skips = 0.0 # percentage of examples with missing data before ignoring a conjecture
use_operators =  [ '-1', '+1', '*2', '/2', '^2', '-()', '1/',
                  'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil',
                  'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^']
#use_operators =  [ '-1', '+1', '*2']

print("Reading data! \n")
sys.stdout.flush()

house_df = pd.read_csv(my_path + 'trainData.csv')

print("Done reading data! \n")

house_df.columns = house_df.columns.str.replace("\\.", "_", regex=True)

house_df["cutoff"] = 300000

invariant_names = ['BEDS',
                   'BATHS',
                   'SQUARE_FEET',
                   'LOT_SIZE',
                   'YEAR_BUILT',
                   'DAYS_ON_MARKET',
                   'HOA_MONTH',
                   'X__SQUARE_FEET',
                   'LATITUDE',
                   'LONGITUDE',
                   'cutoff'
                   ]
categorical_names = ['TARGET',
                     'PROPERTY_TYPEMobile_Manufactured_Home',
                     'PROPERTY_TYPESingle_Family_Residential',
                     'PROPERTY_TYPETownhouse',
                     'PROPERTY_TYPEMulti_Family__2_4_Unit_',
                     'PROPERTY_TYPEMulti_Family__5__Unit_',
                     'PROPERTY_TYPEOther'
                     ]

target = 'TARGET'

my_data = house_df


if target in categorical_names:
    categorical_names[categorical_names.index(target)] = "TARGET"
else:
    invariant_names[invariant_names.index(target)] = "TARGET"

my_data = my_data.rename(columns={target: "TARGET"})
my_data = my_data[invariant_names + categorical_names]

Reading data! 

Done reading data! 



Convert categorical variables to dummies.  One dummy for each binary variable and one dummy for each level for variables with more than two levels.

In [98]:
(my_df, property_names, target_property_names) = create_dummies(my_data,
                                                           categorical_names,
                                                           nan_is_level=True)

Define class, invariants, properties, and target properties (if applicable).

In [99]:
Example = create_example_class(my_df,
                     invariant_names,
                     property_names,
                     categorical_names,
                     target_property_names)

Split into training and testing data.

In [100]:
if "TARGET" in categorical_names:
    X_train, X_test = train_test_split(
        my_df.index,
        stratify=my_data["TARGET"],  # stratify on target levels
        train_size=num_train,
        random_state=12345
    )
else:
    X_train, X_test = train_test_split(
        my_df.index,
        train_size=num_train,
        random_state=12345
    )

Create examples for conjecturing.

In [101]:
train_examples = [Example(i, my_df) for i in X_train]
test_examples = [Example(i, my_df) for i in X_test]

Get lists of invariant and property functions.

In [102]:
(invariants, properties, target_properties) = (
        get_invariants_properties(Example,
                                  invariant_names,
                                  property_names,
                                  categorical_names,
                                  target_property_names)
                                               )

#print(properties)
#for j in train_examples:
#    print(properties[0].__name__)
#    print(properties[0](j))

Invariant conjecturing - upper and lower bounds.

In [103]:
inv_file = open(my_path + inv_fname, "w")
inv_conjectures = invariant_conjecturing(Example,
                                         train_examples,
                                         categorical_names,
                                         target_property_names,
                                         invariants,
                                         use_operators,
                                         complexity,
                                         my_time,
                                         my_skips,
                                         inv_file,
                                         my_path=my_path)

## Evaluate a conjecture
#for j in train_examples:
#    print(inv_conjectures[3].__name__)
#    print(j.SQUARE_FEET(), inv_conjectures[3].evaluate(j, returnBoundValue=True))
#   # print(house_above_properties[0].evaluate(j, returnBoundValue=True), j.number_of_square_feet())

TARGET_0
0 1 2 3 4 5 6 7 8 9 10 
number of conjectures 62
TARGET_0 False
0 1 2 3 4 5 6 7 8 9 10 
number of conjectures 198


Property conjecturing - sufficient conditions for a categorical target values.  For a binary target, get sufficient conditions for the positive class and necessary conditions for the negative class.

In [104]:
prop_file = open(my_path + prop_fname, "w")
(prop_conjs, conditions) = property_conjecturing(Example,
                                                 properties,
                                                 inv_conjectures,
                                                 categorical_names,
                                                 target_property_names,
                                                 train_examples,
                                                 my_time,
                                                 my_skips,
                                                 prop_file,
                                                 my_path=my_path)

#print("conditions")
#print(conditions["TARGET_0"][0].__name__)
#for j in train_examples:
#    print(j.SQUARE_FEET(), conditions["TARGET_0"][0](j))
#print(conditions["necessary"][0].__name__)
#for j in train_examples:
#    print(j.SQUARE_FEET(), conditions["necessary"][0](j))


#for j in train_examples:
#    print(prop_conjs[0].__name__)
#    print(j.BEDS(), j.SQUARE_FEET(), prop_conjs[0].evaluate(j))



TARGET_0
TARGET_0 Necessary


Apply property conjectures to train and test data if target is categorical.

In [105]:
(X_train_df, X_test_df, y_train_df, y_test_df) =  (
        apply_property_conjectures(my_data,
                                   my_df,
                                   X_train,
                                   X_test,
                                   property_names,
                                   invariant_names,
                                   categorical_names,
                                   target_property_names,
                                   conditions,
                                   train_examples,
                                   test_examples)
        )



Calculate support, precision, recall, lift, and F1.  The F1 score is only for the class for the sufficient condition it was derived for.

In [106]:
results_df = evaluate_property_conjectures(categorical_names,
                              target_property_names,
                              Example,
                              conditions,
                              test_examples,
                              y_test_df)

print(results_df)

       class  support  precision    recall      lift        f1
0   TARGET_0        4   1.000000  0.012987  3.217532  0.025641
1   TARGET_0      374   0.745989  0.905844  2.400245  0.818182
2  necessary      152   0.881579  0.435065  1.279128  0.320958
3  necessary      421   0.833729  1.139610  1.209701  0.635870
4  necessary      132   0.553030  0.237013  0.802420  0.179141
